### Instruction of Installing `webdriver` required by `selenium `
1. install `selenium` package
2. download `webdriver` that matches your browser
<div>Exampe: I'm using Chrome 74.0.3729 on Linux. So I download driver from <a href='https://chromedriver.storage.googleapis.com/index.html?path=74.0.3729.6/'>https://chromedriver.storage.googleapis.com/index.html?path=74.0.3729.6/</a><div>
        If you are using chrome as well, you can find your chrome version here:

    <img src="https://help.zenplanner.com/hc/article_attachments/360035466454/_d41a4d465585af6b317d741be1b103c2__Image_2019-05-22_at_8.00.41_AM.png" width="100%">
    
<br/><br/>
<br/><br/>
<div><font size='3'>And find your chromedriver matching your version </font><a href='https://chromedriver.storage.googleapis.com/index.html'>here https://chromedriver.storage.googleapis.com/index.html</a><div>

## Imports / Libraries

In [ ]:
from bs4 import BeautifulSoup 
from time import sleep 
import pandas as pd 
from selenium import webdriver 
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Macro definition - Webscraping

In [ ]:
def get_page(page):
    url = f"https://www.iaaf.org/world-rankings/overall-ranking/men?regionType=world&page={page}&rankDate=2019-08-27&limitByCountry=0"
    option = webdriver.ChromeOptions().add_argument('--incognito')
    browser = webdriver.Chrome(executable_path='./chromedriver', options=option)  
    browser.get(url)
    sleep(5)
    page_source = browser.page_source
    soup = BeautifulSoup(page_source, 'html5lib')
    browser.close()
    return soup


## Loop - Macro execution

In [ ]:
iaaf_database = pd.DataFrame()

for i in range(121):
    temp = get_page(i+1)
    
    interesting_tds = ['Rank', 'Competitor', 'DOB', 'Nat', 'score', 'EventList']
    Rank = list()
    Competitor = list()
    DOB = list()
    Nat = list()
    Score = list()
    Eventlist = list()

    for td in temp.find_all("td"):
        if td.get('data-th') == 'Rank':
            Rank.append(td.text.strip())

    for td in temp.find_all("td"):
        if td.get('data-th') == 'Competitor':
            Competitor.append(td.text.strip())
        
    for td in temp.find_all("td"):
        if td.get('data-th') == 'DOB':
            DOB.append(td.text.strip())
        
    for td in temp.find_all("td"):
        if td.get('data-th') == 'Nat':
            Nat.append(td.text.strip())
        
    for td in temp.find_all("td"):
        if td.get('data-th') == 'score':
            Score.append(td.text.strip())
        
    for td in temp.find_all("td"):
        if td.get('data-th') == 'EventList':
            Eventlist.append(td.text.strip())
            
    data_athletes = {'Rank':Rank,'Competitor':Competitor,'DOB':DOB,'Nat':Nat,'Score':Score,'Eventlist':Eventlist}
    data_athletes_df = pd.DataFrame(data=data_athletes)
    iaaf_database = iaaf_database.append(data_athletes_df)
    
    # Printing dataframe length and the page # just done, for checking
    print(len(iaaf_database))
    print("page", i+1, "done")

#Storing the monthly data into a csv
iaaf_database["DateRank"] = datetime.date(2019, 8, 27)
iaaf_database.to_csv(r'C:\Users\Victor Leo\iaaf_database_20190827.csv')

## Compiling monthly scores into one dataset

In [ ]:
january = pd.read_csv("C:/Users/Victor Leo/Desktop//iaaf_database_20190129.csv", parse_dates=['DOB','DateRank'])
february = pd.read_csv("C:/Users/Victor Leo/Desktop//iaaf_database_20190226.csv", parse_dates=['DOB','DateRank'])
march = pd.read_csv("C:/Users/Victor Leo/Desktop//iaaf_database_20190326.csv", parse_dates=['DOB','DateRank'])
april = pd.read_csv("C:/Users/Victor Leo/Desktop//iaaf_database_20190430.csv", parse_dates=['DOB','DateRank'])
may = pd.read_csv("C:/Users/Victor Leo/Desktop//iaaf_database_20190528.csv", parse_dates=['DOB','DateRank'])
june = pd.read_csv("C:/Users/Victor Leo/Desktop//iaaf_database_20190625.csv", parse_dates=['DOB','DateRank'])
july = pd.read_csv("C:/Users/Victor Leo/Desktop//iaaf_database_20190730.csv", parse_dates=['DOB','DateRank'])
august = pd.read_csv("C:/Users/Victor Leo/Desktop//iaaf_database_20190827.csv", parse_dates=['DOB','DateRank'])

# Appending all datasets
complete = pd.concat([january,february,march,april,may,june,july,august], sort=False)
complete = complete[["Rank","Competitor","DOB","Nat","Score","Eventlist","DateRank"]]

# Calculating the age in years
complete["age"] = complete["DateRank"] - complete["DOB"]
complete["age"] = complete["age"]/np.timedelta64(1,'Y')

# Calculating the number of events
complete['Events']=complete['Eventlist'].str.split('[').str[0]
complete['Events']=complete['Events'].str.replace(",000", "000")
complete["number_events"] = complete["Events"].str.count(',') + 1
